In [10]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl

In [11]:
!pip3 install torchvision

In [12]:
!pip install tensorboard
!pip install tensorboardX

# Installation
## Install from Source
Pyro supports Python 2.7.* and Python 3. To setup, install PyTorch then run:

In [13]:
!pip3 install pyro-ppl

In [0]:
class args(object):
  def __init__(self):
    self.cuda = False
    self.epoch = 2
    self.test_frequency = 0
    
args = args()

In [5]:
from __future__ import print_function
import numpy as np
import torch
from torch.autograd import Variable
import pyro
from pyro.optim import Adam
from pyro.infer import SVI
import pyro.distributions as dist

# clear the param store in case we're in a REPL
pyro.clear_param_store()

# create some data with 6 observed heads and 4 observed tails
data = []
for _ in range(6):
    data.append(Variable(torch.ones(1)))
for _ in range(4):
    data.append(Variable(torch.zeros(1)))

def model(data):
    # define the hyperparameters that control the beta prior
    alpha0 = Variable(torch.Tensor([10.0]))
    beta0 = Variable(torch.Tensor([10.0]))
    # sample f from the beta prior
    f = pyro.sample("latent_fairness", dist.beta, alpha0, beta0)
    # loop over the observed data
    for i in range(len(data)):
        # observe datapoint i using the bernoulli likelihood
        pyro.observe("obs_{}".format(i), dist.bernoulli, data[i], f)

def guide(data):
    # define the initial values of the two variational parameters
    # we initialize the guide near the model prior (except a bit sharper)
    log_alpha_q_0 = Variable(torch.Tensor([np.log(15.0)]), requires_grad=True)
    log_beta_q_0 = Variable(torch.Tensor([np.log(15.0)]), requires_grad=True)
    # register the two variational parameters with Pyro
    log_alpha_q = pyro.param("log_alpha_q", log_alpha_q_0)
    log_beta_q = pyro.param("log_beta_q", log_beta_q_0)
    alpha_q, beta_q = torch.exp(log_alpha_q), torch.exp(log_beta_q)
    # sample latent_fairness from Beta(alpha_q, beta_q)
    pyro.sample("latent_fairness", dist.beta, alpha_q, beta_q)

# setup the optimizer
adam_params = {"lr": 0.0005, "betas": (0.90, 0.999)}
optimizer = Adam(adam_params)

# setup the inference algorithm
svi = SVI(model, guide, optimizer, loss="ELBO", num_particles=7)

n_steps = 4000
# do gradient steps
for step in range(n_steps):
    svi.step(data)
    if step % 100 == 0:
        print('.', end='')

# grab the learned variational parameters
alpha_q = torch.exp(pyro.param("log_alpha_q")).data.numpy()[0]
beta_q = torch.exp(pyro.param("log_beta_q")).data.numpy()[0]

# here we use some facts about the beta distribution
# compute the inferred mean of the coin's fairness
inferred_mean = alpha_q / (alpha_q + beta_q)
# compute inferred standard deviation
factor = beta_q / (alpha_q * (1.0 + alpha_q + beta_q))
inferred_std = inferred_mean * np.sqrt(factor)

print("\nbased on the data and our prior belief, the fairness " +
      "of the coin is %.3f +- %.3f" % (inferred_mean, inferred_std))

........................................
based on the data and our prior belief, the fairness of the coin is 0.536 +- 0.089


# SVI Part I: An Introduction to Stochastic Variational Inference in Pyro

Pyro has been designed with particular attention paid to supporting stochastic variational inference as a general purpose inference algorithm. Let’s see how we go about doing variational inference in Pyro.

Pyro는 범용 추론 알고리즘으로서 확률적 추론을 지원하는데 특히 주의를 기울여 설계되었습니다. Pyro에서 어떻게 변이유추를 하는지에 대해 살펴 보겠습니다.

## Setup

We’re going to assume we’ve already defined our model in Pyro (for more details on how this is done see Intro Part I). As a quick reminder, the model is given as a stochastic function model(*args, **kwargs), which, in the general case takes arguments. The different pieces of model() are encoded via the mapping:

파이로 (Pyro)에서 모델을 이미 정의했다고 가정합니다 (자세한 내용은 소개 섹션 I을 참조하십시오). 이 모델은 확률적 함수모델(* args, ** kwargs)로 주어지며, 일반적으로 arguments를 취합니다. model()의 다른 부분은 매핑을 통해 인코딩됩니다.

1. observations ⟺ pyro.observe
2. latent random variables ⟺ pyro.sample
3. parameters ⟺ pyro.param

Now let’s establish some notation. The model has observations $x$ and latent random variables $z$ as well as parameters $\theta$. It has a joint probability density of the form

이제 몇 가지 표기법을 설정해 보겠습니다. 모델은 관측 $x$와 잠재 임의 변수 $z$와 매개 변수 $\theta$를가집니다. 모델은 다음의 결합 확률 밀도를 가지고있습니다.

$p_{\theta}({\bf x}, {\bf z}) = p_{\theta}({\bf x}|{\bf z}) p_{\theta}({\bf z})$

We assume that the various probability distributions $p_{i}$ that make up $p_{\theta}(x,z)$ have the following properties:

$p_{\theta}({\bf x}, {\bf z})$를 구성하는 다양한 확률 분포 $p_{i}$는 다음과 같은 성질을 가지고 있다고 가정합니다 :

1. we can sample from each $p_{i}$
2. we can compute the pointwise log pdf $p_{i}$
3. $p_{i}$ is differentiable with respect to the parameters $\theta$


1. $p_{i}$로 부터 샘플링합니다.
2. pointwise log pdf $p_{i}$를 계산할 수 있습니다.
3. $p_{i}$는 매개 변수 $\theta$에 대하여 미분 할 수있습니다.


## Model Learning

In this context our criterion for learning a good model will be maximizing the log evidence, i.e. we want to find the value of $\theta$ given by

이러한 맥락에서 좋은 모델을 학습하는 우리의 기준은 log evidence를 최대화하는 것입니다. 즉, 다음에 의해 주어진 $\theta$의 값을 찾고 싶습니다.

$\theta_{\rm{max}} = \underset{\theta}{\operatorname{argmax}} \log p_{\theta}({\bf x})$

where the log evidence $\log p_{\theta}({\bf x})$ is given by 

여기서 log evidence $\log p_{\theta}({\bf x})$는 다음과 같이 나타납니다.

$\log p_{\theta}(x) = \log \int\! d{\bf z}\; p_{\theta}({\bf x}, {\bf z})$

In the general case this is a doubly difficult problem. This is because (even for a fixed $\theta$) the integral over the latent random variables ${\bf z}$ is often intractable. Conversely, even if we know how to calculate the log evidence for all values of $\theta$, maximizing the log evidence as a function of $\theta$ will in general be a difficult non-convex optimization problem.

일반적인 경우 이것은 매우 어려운 문제입니다. 이것은 고정 된 $\theta$에 대해서도 잠재 랜덤 변수 ${\bf z}$에 대한 적분은 intractable 기 때문입니다. 반대로, 우리가 $\theta$의 모든 값에 대한 log evidence를 계산하는 방법을 알고 있더라도 log evidence를 $\theta$의 함수로 최대화하는 것은 일반적으로 어려운 비-볼록 최적화 문제가됩니다.

In addition to finding $\theta_{max}$, we would like to calculate the posterior over the latent variables ${\bf z}$:

$\theta_{max}$를 찾는 것 외에도 잠재 변수 ${\bf z}$에 대한 posterior를 계산하고 싶습니다:

$p_{\theta_{\rm{max}}}({\bf z} | {\bf x}) = \frac{p_{\theta_{\rm{max}}}({\bf x} , {\bf z})}{\int \! d{\bf z}\; p_{\theta_{\rm{max}}}({\bf x} , {\bf z}) }$

Note that the denominator of this expression is the (usually intractable) evidence. Variational inference offers a scheme for finding $\theta_{max}$ and computing an approximation to the posterior $P_{\theta_{max}}({\bf z}|{\bf x})$. Let’s see how that works.

이 표현식의 분모는 (보통 다루기 힘든) evidence라는 점에 유의하십시오. Variational inference는 $\theta_{max}$를 찾고 후미 $P_{\theta_{max}}({\bf z}|{\bf x})$로 근사치를 계산하는 체계를 제공합니다. 어떻게 작동하는지 봅시다.

## Guide
The basic idea is that we introduce a parameterized distribution $q_{\phi}({\bf z})$, where ${\bf \phi}$ are known as the variational parameters. This distribution is called the variational distribution in most of the literature, and in the context of Pyro it’s called the guide (one syllable instead of nine!). The guide will serve as an approximation to the posterior.

기본 개념은 매개 parameterized distribution $q_{\phi}({\bf z})$를 소개하는 것입니다. 여기서 ${\bf \phi}$는 variational parameters로 알려져 있습니다. 이 분포는 대부분의 문헌에서 variational distribution라고 불리며, Pyro에서는 guide(9 대신 한 음절!)라고합니다. guide는 posterior에 대한 근사치를 제공합니다.

Just like the model, the guide is encoded as a stochastic function guide() that contains pyro.sample and pyro.param statements. It does not contain pyro.observe statements, since the guide needs to be a properly normalized distribution. Note that Pyro enforces that model() and guide() have the same call signature, i.e. both callables should take the same arguments.

모델과 마찬가지로 가이드는 pyro.sample 및 pyro.param 문을 포함하는 확률적 함수 guide()로 인코딩됩니다. 이 안내서는 pyro.observe 문을 포함하고 있지 않습니다. 왜냐하면 guide는 적절히 정규화 분포가 되어야하기 때문입니다. Pyro는 model()과 guide()가 동일한 호출시그니처를 갖도록 요구합니다. 즉, 두 호출은 모두 동일한 arguments를 가져야합니다.

Since the guide is an approximation to the posterior $p_{\theta_{max}}({\bf z}|{\bf x})$, the guide needs to provide a valid joint probability density over all the latent random variables in the model. Recall that when random variables are specified in Pyro with the primitive statement pyro.sample() the first argument denotes the name of the random variable. These names will be used to align the random variables in the model and guide. To be very explicit, if the model contains a random variable z_1

guide는 posterior $p_{\theta_{max}}({\bf z}|{\bf x})$에 근사하기 때문에 guide는 모든 latent 랜덤 변수에 대해 유효한 결합 확률 밀도를 제공해야합니다. 모델. Pyro에서 primitive statement pyro.sample()을 사용하여 랜덤 변수를 지정하면 첫 번째 인수는 랜덤 변수의 이름을 나타냅니다. 이 이름들은 모델과 guide의 임의 변수를 정렬하는 데 사용됩니다. 모델에 랜덤 변수 z_1을 포함하는 경우 매우 명확하게 나타 내기 위해

In [0]:
def model():
    pyro.sample("z_1", ...)

then the guide needs to have a matching sample statement

guide에는 일치하는 sample 문이 있어야합니다.

In [0]:
def guide():
    pyro.sample("z_1", ...)

The distributions used in the two cases can be different, but the names must line-up 1-to-1.

Once we’ve specified a guide (we give some explicit examples below), we’re ready to proceed to inference. Learning will be setup as an optimization problem where each iteration of training takes a step in $\theta - \phi$ space that moves the guide closer to the exact posterior. To do this we need to define an appropriate objective function.

두 경우에 사용되는 분포들은 다를 수 있지만 이름은 일대일로 나열 되어야합니다.

guide를 지정했으면 (아래에 몇 가지 명시적인 예제가 있음) inference를 진행할 준비가되었습니다. 학습은 최적의 문제로 설정 될 것인데, $\theta - \phi$ 공간 안에서, 각 반복 학습은 guide를 정확한 posterior에 더 가깝게 이동시킵니다.. 이를 위해서는 적절한 목적 함수를 정의해야합니다.

## ELBO
A simple derivation (for example see reference [1]) yields what we’re after: the evidence lower bound (ELBO). The ELBO, which is a function of both $\theta$ and $\phi$, is defined as an expectation with repect to samples from the guide:

간단한 도함수(예제는 참고 문헌[1] 참조)는 다음의 결과를 산출합니다 : 증거 하한값 (ELBO). $\theta$와 $\phi$의 함수 인 ELBO는 guide로 부터 샘플에 대한 기댓값으로 정의됩니다.

${\rm ELBO} \equiv \mathbb{E}_{q_{\phi}({\bf z})} \left [\log p_{\theta}({\bf x}, {\bf z}) - \log q_{\phi}({\bf z})\right]$

By assumption we can compute the log probabilities inside the expectation. And since the guide is assumed to be a parametric distribution we can sample from, we can compute Monte Carlo estimates of this quantity. Crucially, the ELBO is a lower bound to the log evidence, i.e. for all choices of $\theta$ and $\phi$ we have that

가정을 통해 기댓값 내에서 로그 확률을 계산할 수 있습니다. 그리고 guide는 우리가 샘플링 할 수있는 parametric 분포로 가정되므로,이 양에 대한 몬테카를로 추정을 계산할 수 있습니다. 결정적으로, ELBO는 log evidence의 하한값, 즉 $\theta$ 및 $\phi$의 모든 선택에 대해

$\log p_{\theta}({\bf x}) \ge {\rm ELBO}$

So if we take (stochastic) gradient steps to maximize the ELBO, we will also be pushing the log evidence higher (in expectation). Furthermore, it can be shown that the gap between the ELBO and the log evidence is given by the KL divergence between the guide and the posterior:

따라서 ELBO를 최대화하기 위해 (확률적으로) 그래디언트 단계를 취하면, log evidence를 더 높게 (기대값 안에서) 밀어 낼 것입니다. 또한, ELBO와 log evidence 사이의 갭은 guide와 posterior 사이의 KL 발산에 의해 제공된다는 것을 보여줄 수있다:

$\log p_{\theta}({\bf x}) - {\rm ELBO} =\rm{KL}\!\left( q_{\phi}({\bf z}) \lVert p_{\theta}({\bf z} | {\bf x}) \right)$

This KL divergence is a particular (non-negative) measure of ‘closeness’ between two distributions. So, for a fixed $\theta$, as we take steps in $\phi$ space that increase the ELBO, we decrease the KL divergence between the guide and the posterior, i.e. we move the guide towards the posterior. In the general case we take gradient steps in both $\theta$ and $\phi$ space simultaneously so that the guide and model play chase, with the guide tracking a moving posterior $log{\bf p_{\theta}}({\bf z}|{\bf x})$. Perhaps somewhat surprisingly, despite the moving target, this optimization problem can be solved (to a suitable level of approximation) for many different problems.

이 KL 발산은 두 분포 사이의 '근접성'에 대한 특별한 (부정적이지 않은) 척도입니다. 따라서 고정 된 $\theta$에 대해 ELBO를 증가시키는 $\phi$ 공간에서, guide와 posterior 사이의 KL 발산을 줄입니다. 즉 guide를 posterior쪽으로 이동시킵니다. guide는 변화하는 posterior $log{\bf p_{\theta}}({\bf z}|{\bf x})$을 따라가면서, 일반적인 경우 $\theta$와 $\phi$ 공간에서 그라디언트 단계를 동시에 수행하여 guide와 model이 업데이트됨. 다소 놀랍게도, 움직이는 목표에도 불구하고이 최적화 문제는 많은 다른 문제에 대해 (적절한 근사 수준으로) 해결 될 수 있습니다.

So at high level variational inference is easy: all we need to do is define a guide and compute gradients of the ELBO. Actually, computing gradients for general model and guide pairs leads to some complications (see the tutorial SVI Part III for a discussion). For the purposes of this tutorial, let’s consider that a solved problem and look at the support that Pyro provides for doing variational inference.

그래서 높은 수준의 VI는 쉽습니다. 우리가 해야 할 일은 ELBO의 가이드와 그라디언트를 정의하는 것입니다. 사실, 일반적인 모델 및 가이드 쌍의 그라디언트를 계산하면 몇 가지 문제가 발생합니다(이에 대한 설명은 SVI 파트 III 참조). 이 튜토리얼의 목적을 위해, 해결 된 문제와 Pyro가 variational Inferences를 수행하기 위해 제공하는 지원을 살펴 보겠습니다.

## SVI Class

In Pyro the machinery for doing variational inference is encapsulated in the SVI class. (At present SVI only provides support for the ELBO objective, but in the future Pyro will provide support for alternative variational objectives.)

Pyro에서 VI를 수행하는 메커니즘은 SVI 클래스에 캡슐화되어 있습니다. (현재 SVI는 ELBO objective만을 지원하지만, 앞으로 Pyro는 alternative variational objectives를 지원할 것입니다.)

The user needs to provide three things: the model, the guide, and an optimizer. We’ve discussed the model and guide above and we’ll discuss the optimizer in some detail below, so let’s assume we have all three ingredients at hand. To construct an instance of SVI that will do optimization via the ELBO objective, the user writes

사용자는 세 가지를 제공해야합니다 : model, guide 및 optimizer. 위에서는 model과 guide를 살펴보았고, optimizer에 대해 아래에서 자세히 설명하겠습니다. 따라서 세 가지 요소가 모두 갖추어져 있다고 가정 해 봅시다. ELBO objective을 통해 최적화를 수행 할 SVI의 인스턴스를 생성하기 위해 사용자는 다음을 작성합니다.

In [0]:
import pyro
from pyro.infer import SVI
svi = SVI(model, guide, optimizer, loss="ELBO")

The SVI object provides two methods, step() and evaluate_loss(), that encapsulate the logic for variational learning and evaluation:

1. The method step() takes a single gradient step and returns an estimate of the loss (i.e. minus the ELBO). If provided, the arguments to step() are piped to model() and guide().
2. The method evaluate_loss() returns an estimate of the loss without taking a gradient step. Just like for step(), if provided, arguments to evaluate_loss() are piped to model() and guide().

For the case where the loss is the ELBO, both methods also accept an optional argument num_particles, which denotes the number of samples used to compute the loss (in the case of evaluate_loss) and the loss and gradient (in the case of step). Note that SVI also provides support for user-defined losses; see the documentation for details.

SVI 객체는 variational 학습 및 평가를 위한 logic을 캡슐화하는 두 가지 메소드 인 step() 및 evaluate_loss()를 제공합니다:

1. 메소드 step()은 단일 그래디언트 단계를 취하여 loss의 estimate(즉, 음의 ELBO)를 반환합니다. step()에 대한 인수가 제공된다면, step()에 대한 인수는 model() 및 guide()에 파이프됩니다.
2. evaluate_loss() 메소드는 그래디언트 단계를 거치지 않고 loss의 estimate를 반환합니다. step()과 마찬가지로 evaluate_loss()에 대한 인수가 제공되면 model() 및 guide()에 파이프됩니다.

loss가 ELBO 인 경우 두 method 모두 num_particles 옵션을 사용할 수 있습니다.(loss(evaluate_loss의 경우)와 loss 및 그래디언트(step의 경우)를 계산하기 위해 사용되는). SVI는 또한 사용자 정의 loss들을 지원합니다. 자세한 내용은 설명서를 참조하십시오.

## Optimizers
In Pyro, the model and guide are allowed to be arbitrary stochastic functions provided that

1. guide doesn’t contain any pyro.observe statements
2. model and guide have the same call signature

This presents some challenges because it means that different executions of model() and guide() may have quite different behavior, with e.g. certain latent random variables and parameters only appearing some of the time. Indeed parameters may be created dynamically during the course of inference. In other words the space we’re doing optimization over, which is parameterized by $\theta$ and $\phi$, can grow dynamically.

In order to support this behavior, Pyro needs to dynamically generate an optimizer for each parameter the first time it appears during learning. Luckily, PyTorch has a lightweight optimization library (see torch.optim) that can easily be repurposed for the dynamic case.

All of this is controlled by the optim.PyroOptim class, which is basically a thin wrapper around PyTorch optimizers. PyroOptim takes two arguments: a constructor for PyTorch optimizers optim_constructor and a specification of the optimizer arguments optim_args. At high level, in the course of optimization, whenever a new parameter is seen optim_constructor is used to instantiate a new optimizer of the given type with arguments given by optim_args.

Most users will probably not interact with PyroOptim directly and will instead interact with the aliases defined in optim/__init__.py. Let’s see how that goes. There are two ways to specify the optimizer arguments. In the simpler case, optim_args is a fixed dictionary that specifies the arguments used to instantiate PyTorch optimizers for all the parameters:

Pyro에서 model과 guide는 임의의 확률적 함수가 될 수 있습니다.

1. guide는 pyro.observe 문을 포함하지 않습니다.
2. model과 guide는 동일한 call signature를 가지고 있습니다.

이는 model ()과 guide ()의 서로 다른 실행이 매우 다른 동작을 가질 수 있다는 것을 의미하기 때문에 몇 가지 문제점을 제시합니다. 특정 latent 랜덤 변수와 매개 변수는 일부 시간에만 나타납니다. 실제로 매개 변수는 inference 중에 동적으로 생성 될 수 있습니다. 즉, $\theta$ 및 $\phi$로 매개 변수화 된 최적화 작업을 수행하는 공간이 동적으로 커질 수 있습니다.

이 동작을 지원하기 위해, Pyro는 처음 배우기 시작할 때 각 매개 변수에 대한 optimizer를 동적으로 생성해야합니다. 운 좋게도, PyTorch는 동적 인 경우 쉽게 용도 변경이 가능한 경량 최적화 라이브러리 (torch.optim 참고)를 가지고 있습니다.

이 모든 것은 optim.PyroOptim 클래스에 의해 제어됩니다. 이 클래스는 기본적으로 PyTorch 최적화 알고리즘을 감싸는 얇은 래퍼입니다. PyroOptim은 PyTorch 옵티마이저 optim_constructor의 생성자와 옵티 마이저 인수 optim_args의 스펙이라는 두 가지 인수를 취합니다. high level의 최적화 과정에서 새로운 매개 변수가 발견 될 때마다 optim_constructor는 optim_args에 의해 주어진 인수를 사용하여 지정된 유형의 새로운 최적화 프로그램을 인스턴스화하는 데 사용됩니다.

대부분의 사용자는 PyroOptim과 직접 상호 작용하지 않으며 대신 optim/__init__.py에 정의 된 별칭과 상호 작용합니다. 그게 어떻게되는지 봅시다. 옵티마이저 인수를 지정하는 두 가지 방법이 있습니다. 간단한 경우 optim_args는 모든 매개 변수에 대해 PyTorch optimizers를 인스턴스화하는데 사용되는 인수를 지정하는 fixed dictionary입니다.

In [0]:
from pyro.optim import Adam

adam_params = {"lr": 0.005, "betas": (0.95, 0.999)}
optimizer = Adam(adam_params)

The second way to specify the arguments allows for a finer level of control. Here the user must specify a callable that will be invoked by Pyro upon creation of an optimizer for a newly seen parameter. This callable must have the following signature:

1. module_name: the Pyro name of the module containing the parameter, if any
2. param_name: the Pyro name of the parameter
3. tags: a (possibly empty) iterable of parameter tags

This gives the user the ability to, for example, customize learning rates for different parameters. For an example where this sort of level of control is useful, see the discussion of baselines. Here’s a simple example to illustrate the API:

인수를 지정하는 두번째 방법은 더 세밀한 제어 레벨을 허용합니다. 여기서 사용자는 새로 조회한 매개 변수에 대한 optimizer 작성시 Pyro에 의해 호출 될 callable을 지정해야합니다. 이 callable 객체에는 다음 signature가 있어야합니다:

1. module_name : 매개 변수를 포함하는 모듈의 Pyro 이름 (있는 경우)
2. param_name : 매개 변수의 Pyro 이름
3. tags : 매개 변수 태그의 iterable (비워놓는 것도 가능)

이를 통해 사용자는 예를 들어 다양한 매개 변수에 대한 learning_rate를 사용자 정의 할 수 있습니다. 이러한 종류의 제어가 유용한 예제는 baselines에 대한 설명을 참조하십시오. 다음은 API를 설명하기위한 간단한 예입니다:

In [0]:
from pyro.optim import Adam

def per_param_callable(module_name, param_name, tags):
    if 'param_name' == 'my_special_parameter':
        return {"lr": 0.010}
    else:
        return {"lr": 0.001}

optimizer = Adam(per_param_callable)

This simply tells Pyro to use a learning rate of 0.010 for the Pyro parameter my_special_parameter and a learning rate of 0.001 for all other parameters.

이것은 단순히 Pyro에게 Pyro 매개 변수 my_special_parameter에 대해 0.010의 학습 속도를 사용하고 다른 모든 매개 변수에 대해 0.001의 학습 속도를 사용하도록 알려줍니다.

## A simple example
We finish with a simple example. You’ve been given a two-sided coin. You want to determine whether the coin is fair or not, i.e. whether it falls heads or tails with the same frequency. You have a prior belief about the likely fairness of the coin based on two observations:

* it’s a standard quarter issued by the US Mint
* it’s a bit banged up from years of use

So while you expect the coin to have been quite fair when it was first produced, you allow for its fairness to have since deviated from a perfect 1:1 ratio. So you wouldn’t be surprised if it turned out that the coin preferred heads over tails at a ratio of 11:10. By contrast you would be very surprised if it turned out that the coin preferred heads over tails at a ratio of 5:1—it’s not that banged up.

To turn this into a probabilistic model we encode heads and tails as 1s and 0s. We encode the fairness of the coin as a real number $f$, where $f$ satisfies $f \in [0.0, 1.0]$ and $f=0.50$ corresponds to a perfectly fair coin. Our prior belief about $f$ will be encoded by a beta distribution, specifically $Beta(10,10)$, which is a symmetric probability distribution on the interval $[0.0,1.0]$ that is peaked at $f=0.5$.

우리는 간단한 예제로 마무리합니다. 당신은 양면 동전을 받았습니다. 당신은 동전이 공정한 것인지 아닌지, 즉 그것이 같은 빈도로 머리를 떨어 뜨릴지 또는 꼬리를 떨어 뜨릴지를 알려고합니다. 당신은 두 가지 관찰에 근거하여 동전의 공정한 가능성에 대한 이전의 믿음을 가지고 있습니다 :

* 미국 조폐국이 발행 한 표준 quarter입니다.
* 몇 년 동안 사용 해왔다.

<img src="https://www.usmint.gov/wordpress/wp-content/uploads/2017/09/2018-america-the-beautiful-quarters-coin-uncirculated-obverse-philadelphia-768x768.jpg" width=150 />

따라서 동전이 처음 생산되었을 때 동전이 상당히 공정하다고 예상한다면, 완벽한 1 : 1 비율에서 벗어난 이후의 공정성을 허용합니다. 따라서 11:10의 비율로 동전이 꼬리보다 머리를 선호하는 것으로 밝혀지면 놀라지 않을 것입니다. 대조적으로, 동전이 꼬리보다 5 : 1의 비율로 머리를 선호하는 것으로 판명되면 매우 놀랄 것입니다.

이것을 확률 론적 모델로 바꾸기 위해 헤드와 테일을 1과 0으로 인코딩합니다. [0.0, 1.0] $ 및 $ f = 0.50 $의 $ f $가 완벽하게 공정한 동전에 해당하는 실수 $ f $로 동전의 공정성을 부호화합니다. $ f $에 대한 우리의 이전 믿음은 $ f = 0.5 $에서 정점에있는 간격 $ [0.0,1.0] $에서 대칭 확률 분포 인 베타 분포, 특히 $ Beta (10,10) $로 인코딩됩니다.
<img src="http://pyro.ai/examples/_static/img/beta.png" width=300 />

Figure 1: The distribution Beta that encodes our prior belief about the fairness of the coin.

To learn something about the fairness of the coin that is more precise than our somewhat vague prior, we need to do an experiment and collect some data. Let’s say we flip the coin 10 times and record the result of each flip. In practice we’d probably want to do more than 10 trials, but hey this is a tutorial.

Assuming we’ve collected the data in a list data, the corresponding model is given by

다소 모호한 pior보다 더 정확한 동전의 공정성에 대해 배우려면 실험을하고 데이터를 수집해야합니다. 우리가 동전을 10 번 뒤집고 각 뒤집기 결과를 기록한다고 가정 해 봅시다. 실제에서는 10 가지 이상의 시험을 원할 것입니다. 그러나 이것은 자습서입니다.

목록 데이터에서 데이터를 수집했다고 가정하면 해당 모델은 다음과 같이 주어진다.

In [0]:
import pyro.distributions as dist

def model(data):
    # define the hyperparameters that control the beta prior
    alpha0 = Variable(torch.Tensor([10.0]))
    beta0 = Variable(torch.Tensor([10.0]))
    # sample f from the beta prior
    f = pyro.sample("latent_fairness", dist.beta, alpha0, beta0)
    # loop over the observed data
    for i in range(len(data)):
        # observe datapoint i using the bernoulli
        # likelihood Bernoulli(f)
        pyro.observe("obs_{}".format(i), dist.bernoulli,
                     data[i], f)

Here we have a single latent random variable ('latent_fairness'), which is distributed according to $Beta(10,10)$. Conditioned on that random variable, we observe each of the datapoints using a bernoulli likelihood. Note that each observation is assigned a unique name in Pyro.

Our next task is to define a corresponding guide, i.e. an appropriate variational distribution for the latent random variable $f$. The only real requirement here is that $q(f)$ should be a probability distribution over the range $[0.0,1.0]$, since $f$ doesn’t make sense outside of that range. A simple choice is to use another beta distribution parameterized by two trainable parameters $\alpha_{q}$ and $\beta_{q}$. Actually, in this particular case this is the ‘right’ choice, since conjugacy of the bernoulli and beta distributions means that the exact posterior is a beta distribution. In Pyro we write:

여기에 우리는 $ Beta (10,10) $에 따라 분포하는 하나의 latent 랜덤 변수 (latent_fairness)를 가지고 있습니다. 그 랜덤 변수를 조건으로, 우리는 베르누이 우도 (bernoulli likelihood)를 사용하여 각각의 데이터 포인트를 관찰합니다. 각 관측치는 Pyro에서 고유 한 이름으로 지정됩니다.

다음 과제는 guide를 정의하는 것입니다, 즉 latent 랜덤 변수 $ f $에 대한 적절한 variational 분포를 정의하는 것입니다. 여기서 유일한 요구 사항은 $ q (f) $가 $ [0.0,1.0] $ 범위의 확률 분포 여야 한다는 것입니다. $ f $가 그 범위를 벗어나면 의미가 없기 때문입니다. 간단한 선택은 학습가능한 파라미터 $\alpha_{q}$와 $\beta_{q}$로 파라미터화된 다른 beta분포를 사용하는 것입니다. 사실이 특별한 경우 베르누이와 베타 분포의 결합은 정확한 posterior가 베타 분포라는 것을 의미하므로 '옳은'선택입니다. Pyro에서 우리는 다음과 같이 씁니다 :

In [0]:
def guide(data):
    # define the initial values of the two variational parameters
    log_alpha_q_0 = Variable(torch.Tensor([np.log(15.0)]),
                             requires_grad=True)
    log_beta_q_0 = Variable(torch.Tensor([np.log(15.0)]),
                            requires_grad=True)
    # register the two variational parameters with Pyro
    log_alpha_q = pyro.param("log_alpha_q", log_alpha_q_0)
    log_beta_q = pyro.param("log_beta_q", log_beta_q_0)
    alpha_q, beta_q = torch.exp(log_alpha_q), torch.exp(log_beta_q)
    # sample latent_fairness from the distribution
    # Beta(alpha_q, beta_q)
    pyro.sample("latent_fairness", dist.beta, alpha_q, beta_q)

There are a few things to note here: - we’ve taken care that the names of the random variables line up exactly between the model and guide - model(data) and guide(data) take the same arguments - the variational parameters are PyTorch Variables with requires_grad=True. if we forget to set the requires_grad flag correctly, the parameters won’t be trained. - the variational parameters are actually defined in log space. this is because we need alpha_q and beta_q to be non-negative in order to define a valid beta distribution.

Now we can proceed to do stochastic variational inference.

여기서 주목해야 할 몇 가지 사항이 있습니다. - 우리는 무작위 변수의 이름이 model과 guide - model(data)과 guide(data) - 사이에 정확히 일치하도록 주의를 기울였습니다. variational 매개 변수는 PyTorch에서 requires_grad=True 인 Variable입니다. requires_grad 플래그를 올바르게 설정하는 것을 잊어 버리면 매개 변수가 훈련되지 않습니다. - 변수 매개 변수는 실제로 log 공간에 정의됩니다. 이는 유효한 베타 분포를 정의하기 위해 alpha_q와 beta_q가 음수가 아니기 위해 필요합니다.

이제 우리는 stochastic variational inference를 진행할 수 있습니다.

In [0]:
# set up the optimizer
adam_params = {"lr": 0.0005, "betas": (0.90, 0.999)}
optimizer = Adam(adam_params)

# setup the inference algorithm
svi = SVI(model, guide, optimizer, loss="ELBO")

n_steps = 5000
# do gradient steps
for step in range(n_steps):
    svi.step(data)

Note that in the step() method we pass in the data, which then get passed to the model and guide.

The only thing we’re missing at this point is some data. So let’s create some data and assemble all the code snippets above into a complete script:

step() 메소드에서 우리는 데이터를 전달하고,이 데이터는 model과 guide로 전달됩니다.

이 시점에서 우리가 놓치고있는 유일한 사실은 일부 데이터입니다. 위의 모든 코드 스니펫을 전체 스크립트로 조합하여 데이터를 작성해 보겠습니다.

In [27]:
from __future__ import print_function
import numpy as np
import torch
from torch.autograd import Variable
import pyro
from pyro.optim import Adam
from pyro.infer import SVI
import pyro.distributions as dist
from tensorboardX import SummaryWriter

writer = SummaryWriter("logs")

# clear the param store in case we're in a REPL
pyro.clear_param_store()

# create some data with 6 observed heads and 4 observed tails
data = []
for _ in range(6):
    data.append(Variable(torch.ones(1)))
for _ in range(4):
    data.append(Variable(torch.zeros(1)))

def model(data):
    # define the hyperparameters that control the beta prior
    alpha0 = Variable(torch.Tensor([10.0]))
    beta0 = Variable(torch.Tensor([10.0]))
    # sample f from the beta prior
    f = pyro.sample("latent_fairness", dist.beta, alpha0, beta0)
    # loop over the observed data
    for i in range(len(data)):
        # observe datapoint i using the bernoulli likelihood
        pyro.observe("obs_{}".format(i), dist.bernoulli, data[i], f)

def guide(data):
    # define the initial values of the two variational parameters
    # we initialize the guide near the model prior (except a bit sharper)
    log_alpha_q_0 = Variable(torch.Tensor([np.log(15.0)]), requires_grad=True)
    log_beta_q_0 = Variable(torch.Tensor([np.log(15.0)]), requires_grad=True)
    # register the two variational parameters with Pyro
    log_alpha_q = pyro.param("log_alpha_q", log_alpha_q_0)
    log_beta_q = pyro.param("log_beta_q", log_beta_q_0)
    alpha_q, beta_q = torch.exp(log_alpha_q), torch.exp(log_beta_q)
    # sample latent_fairness from Beta(alpha_q, beta_q)
    pyro.sample("latent_fairness", dist.beta, alpha_q, beta_q)

# setup the optimizer
adam_params = {"lr": 0.0005, "betas": (0.90, 0.999)}
optimizer = Adam(adam_params)

# setup the inference algorithm
svi = SVI(model, guide, optimizer, loss="ELBO", num_particles=7)

n_steps = 4000
# do gradient steps
for step in range(n_steps):
    loss = svi.step(data)
    writer.add_scalar('Train/loss', loss, step)    
    if step % 100 == 0:
        print('.', end='')
        
# save graph with tensorboardX
writer.close()
        
# grab the learned variational parameters
alpha_q = torch.exp(pyro.param("log_alpha_q")).data.numpy()[0]
beta_q = torch.exp(pyro.param("log_beta_q")).data.numpy()[0]

# here we use some facts about the beta distribution
# compute the inferred mean of the coin's fairness
inferred_mean = alpha_q / (alpha_q + beta_q)
# compute inferred standard deviation
factor = beta_q / (alpha_q * (1.0 + alpha_q + beta_q))
inferred_std = inferred_mean * np.sqrt(factor)

print("\nbased on the data and our prior belief, the fairness " +
      "of the coin is %.3f +- %.3f" % (inferred_mean, inferred_std))

........................................
based on the data and our prior belief, the fairness of the coin is 0.535 +- 0.090


Note that we pass the argument num_particles=7 to SVI so that the ELBO gradient estimator uses seven samples to compute each estimate. We’ll describe another approach for dealing with noisy gradient estimates in another tutorial.

인수 num_particles = 7을 SVI에 전달하여 ELBO 그래디언트 estimator가 각 샘플을 계산하기 위해 7 개의 샘플을 사용합니다. 다른 튜토리얼에서 noisy gradient estimates를 처리하는 또 다른 방법을 설명하겠습니다

In [8]:
LOG_DIR = 'logs/'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

# https://blog.outsider.ne.kr/1159 ngrok으로 로컬 네트워크의 터널 열기
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2018-03-27 23:43:20--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 50.19.121.155, 54.225.179.161, 50.16.250.54, ...
Connecting to bin.equinox.io (bin.equinox.io)|50.19.121.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  9.80MB/s    in 0.5s    

2018-03-27 23:43:21 (9.80 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://a4987007.ngrok.io


In [0]:
!rm -r logs

## Sample output:

In [0]:
based on the data and our prior belief, the fairness of the coin is 0.532 +- 0.090

This estimate is to be compared to the exact posterior mean, which in this case is given by $16/30 = 0.5\bar{3}$. Note that the final estimate of the fairness of the coin is in between the the fairness preferred by the prior (namely 0.50) and the fairness suggested by the raw empirical frequencies (6/10=0.60).

이 추정값은 정확한 posterior 평균과 비교 될 것이며,이 경우에는 $16/30=0.5\bar{3}$로 주어집니다. 동전의 공정성에 대한 최종 추정치는 이전 (0.50)에 의해 선호된 공정성과 원래의 경험 빈도 (6/10 = 0.60)에 의해 제안된 공정성 사이에 있음에 유의하십시오